In [780]:
import pandas as pd
import sys, os
import tensorflow as tf
import shape_helper
from tensorflow import keras
from tensorflow.keras import layers

In [781]:

#the following bits are basically just to deal with translations and rotations in a pretty efficient manner. Note the 
#translation+removal functions have no for loops so it's pretty quick 
#Let's make the goal shape into a bunch of 1's and 0's representing where we want pieces
def mattoones(goalshape1):
    for i in range(len(goalshape1)):
        for k in range(len(np.transpose(goalshape1))):
            if goalshape1[i][k]!='0.0' and goalshape1[i][k]!=0:
                goalshape1[i][k]=1
    return goalshape1
#Remove all rows/columns at edges with only 0's in order to standardize calculations 
def translator(matrix):
    sums=0;
    while(sum(matrix[0])==0):
        if (sum(matrix[0]==0)):
            matrix=np.delete(matrix,0,0)

    i=len(matrix)-1
    while(sums==0):
        sums=sum(matrix[i]);
        if(sums==0):
            matrix=np.delete(matrix,i,0)
        i=i-1
    return(matrix)

#(*use the above function but for columns too*)
def removal(matrixer):
    c=translator(matrixer);
    c=translator(np.transpose(c))
    return(np.transpose(c))
def sqdiff(m1,m2):
    return (sum(sum((m1-m2)*(m1-m2))))
#I think this lets us visualize chains as 1/0s instead of HP but it's not very useful 
def graphicchainno(chain):
    boundary=np.floor(np.sqrt(len(chain))*2);
    lattice=np.ndarray.tolist(np.zeros([int(boundary),int(boundary)]))
    for i in range(len(chain)):
        lattice[chain[i][0]][chain[i][1]]=1
    lattice=np.array(lattice)
    return(lattice)

In [801]:
x=1
model=keras.Sequential([
    #Entry block: create four masks, and run
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same',input_shape=[13,13,1]),
    layers.MaxPool2D(),
    layers.Dropout(.3),
    #Block 2
    layers.Conv2D(filters=128, kernel_size=3,activation="relu",padding='same'),
    layers.MaxPool2D(),layers.Dropout(.3),

    #Third hidden layer
    layers.Conv2D(filters=128,kernel_size=3,activation="relu",padding='same'),layers.Dropout(.3),
    #output 
    layers.Flatten(),
    layers.Dense(units=169,activation="relu"),
    layers.Dense(units=169,activation="sigmoid")
])
model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_123 (Conv2D)         (None, 13, 13, 128)       1280      
                                                                 
 max_pooling2d_82 (MaxPoolin  (None, 6, 6, 128)        0         
 g2D)                                                            
                                                                 
 dropout_32 (Dropout)        (None, 6, 6, 128)         0         
                                                                 
 conv2d_124 (Conv2D)         (None, 6, 6, 128)         147584    
                                                                 
 max_pooling2d_83 (MaxPoolin  (None, 3, 3, 128)        0         
 g2D)                                                            
                                                                 
 dropout_33 (Dropout)        (None, 3, 3, 128)       

1+1

In [791]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')
targets=df['path']
startshape=df['shape_mapping']
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
arraysize=13
#create our array entries 
for i in range(len(startshape)):
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position=start_to_class(d[2],arraysize)
    targets[i]=position
    startshape[i]=mattoones(d[0])

#onehotencode position
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-i])
        validtargets.append(targets[eve-i])
        np.delete(x_train,eve-i,0)
        np.delete(onehottable,eve-i,0)
        
#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (0, -2)
startpos ==> (0, 1)
startpos ==> (0, -2)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, -1)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 

In [784]:
np.mean(targets)

114.04045410937

In [802]:
#set datasets 
#from sklearn.model_selection import train_test_split
import random
import shape_helper
df=pd.read_csv('/Users/joshuatsai/Documents/GitHub/SigmaFold/library/sequence_training.csv')

targets=[]
startshape=[]
lengths=df['length']
tots=len(targets)

        
x_train=[]
y_train=[]
validtargets=[]
validshapes=[]
arraysize=13
#create our array entries 
for i in range(len(lengths)):
    c=shape_helper.deserialize_path(df['path'][i])
    d=path_to_shape(c,arraysize)
    position1=start_to_class(d[2],arraysize)
    position2=start_to_class(d[3],arraysize)
    targets.append(position1)
    targets.append(position2)
    startshape.append(mattoones(d[1]))
    startshape.append(mattoones(d[1]))

#onehotencode position
onehottable=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    onehottable[i][targets[i]]=1
    y_train.append(targets[i])

#onehottablepos=np.zeros([len(targets),arraysize*arraysize])
for i in range(len(targets)):
    x_train.append(startshape[i])
   # for x in range(len(startshape[i])):
       # for y in range(len(startshape[i][1])):
           # if startshape[i][x][y]!=0:
                #onehottablepos[i][x*len(startshape[i]+y)]=1



for i in range(len(startshape)):
    prob=random.random()
    eve=len(startshape)
    if prob<.1:

        validshapes.append(startshape[eve-1-i])
        validtargets.append(targets[eve-1-i])
        np.delete(x_train,eve-i,0)
        np.delete(onehottable,eve-i,0)
        
#startshapes=tf.stack(startshape,np.float32)
onehottableval=np.zeros([len(validtargets),arraysize*arraysize])
for i in range(len(validtargets)):
    onehottableval[i][validtargets[i]]=1
    
trainx=tf.convert_to_tensor(x_train)
trainy=tf.convert_to_tensor(onehottable)
valx=tf.convert_to_tensor(validshapes)
valy=tf.convert_to_tensor(onehottableval)


#trainy=[tf.from_numpy(item).float() for item in startshape]



startpos ==> (0, -2)
startpos ==> (0, 1)
startpos ==> (0, -2)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, -1)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-2, 0)
startpos ==> (-2, -1)
startpos ==> (-1, 2)
startpos ==> (-1, -3)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (-3, -1)
startpos ==> (-3, 0)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 0)
startpos ==> (0, -1)
startpos ==> (0, 

In [803]:
len(x_train)

12508

In [ ]:
np.shape(startshape[120])


(13, 13)

In [ ]:
import math as math
def path_to_shape(path,grid_sizes):
    grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # actual grid for mapping
    temp_grid = np.asarray([[0]*grid_sizes]*grid_sizes, dtype=int) # temp grid to hold the array before alignment
    for coord in path:
        temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 1
        if coord==path[len(path)-1]:
            temp_grid[coord[1]+round(grid_sizes/2)][coord[0]+ round(grid_sizes/2)] = 3
            # path transferred onto grid but uncentered
    temp_grid[round(grid_sizes/2)][round(grid_sizes/2)] = 2  

    temp_grid = np.pad(temp_grid, 1) # zero padding to avoid multiplying by 0 when calculating moments

    # find centroid of temp_grid
    m_00 = len(path) # non-zero residues
    m_01 = 0
    for row_n, row in enumerate(temp_grid):
        if np.any(row!=0):
            m_01 += row_n * np.count_nonzero(row)
    m_10 = 0
    for col_n in range(np.shape(temp_grid)[1]):
        if np.any(temp_grid[:, col_n]!=0):
            m_10 += col_n *  np.count_nonzero(temp_grid[:, col_n])

    # coordinates of centroid
    n_centroid = math.floor((m_10/m_00))
    m_centroid = math.floor((m_01/m_00))
    centroid = (m_centroid, n_centroid)
    #print("centroid ==>", centroid)
    # align temp_grid onto grid
    dev_m = round(grid_sizes/2)-centroid[0]
    dev_n = round(grid_sizes/2)-centroid[1]
    coord_list = np.nonzero(temp_grid)
    print("startpos ==>", (dev_m,dev_n))
    start= (round(grid_sizes/2)+dev_m,round(grid_sizes/2)+dev_n)
    x,y=np.where(temp_grid==3)
    end=(x+dev_m,y+dev_n)
    
    for i in range(len(coord_list[0])):
        grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 1
        #if i==0:
            #grid[coord_list[0][i]+dev_m][coord_list[1][i]+dev_n] = 2
    
    return temp_grid, grid,start,end
def start_to_class(coords,grid_size):
    return coords[0]*grid_size+coords[1]

In [ ]:
trainy

<tf.Tensor: shape=(12508, 169), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [805]:
from tensorflow import keras
from tensorflow.keras import layers
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=.01),
    loss='CategoricalCrossentropy',metrics=['AUC']
)

In [809]:
history = model.fit(
    trainx,trainy,epochs=20
)


Epoch 1/20
391/391 [==============================] - 4s 9ms/step - loss: 1.8072 - auc: 0.9806
Epoch 2/20
391/391 [==============================] - 4s 9ms/step - loss: 1.8000 - auc: 0.9806
Epoch 3/20
391/391 [==============================] - 4s 9ms/step - loss: 1.7976 - auc: 0.9807
Epoch 4/20
391/391 [==============================] - 3s 9ms/step - loss: 1.7955 - auc: 0.9804
Epoch 5/20
391/391 [==============================] - 3s 9ms/step - loss: 1.8028 - auc: 0.9807
Epoch 6/20
391/391 [==============================] - 3s 9ms/step - loss: 1.7939 - auc: 0.9804
Epoch 7/20
391/391 [==============================] - 3s 9ms/step - loss: 1.7964 - auc: 0.9804
Epoch 8/20
391/391 [==============================] - 3s 9ms/step - loss: 1.7942 - auc: 0.9808
Epoch 9/20
391/391 [==============================] - 4s 9ms/step - loss: 1.7935 - auc: 0.9810
Epoch 10/20
391/391 [==============================] - 3s 9ms/step - loss: 1.7922 - auc: 0.9806
Epoch 11/20
391/391 [============================

In [815]:
x=model.predict(valx)

40/40 [==============================] - 0s 3ms/step


In [816]:
thresx

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [817]:
np.where(x[1]==max(x[1]))

(array([57]),)

In [837]:
thresx=threshold(x)
correct=acc(thresx,valy)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [836]:
def acc(array1,array2):
    total=0
    for i in range(len(array1)):
        if len(np.where(array1[i]==1)[0])==1:
            if np.where(array1[i]==1)==np.where(array2[i]==1):
            
                total=1+total
    return total
def threshold(array1):
    lop=np.zeros([len(array1),len(array1[1])])
    for i in range(len(array1)):
        lop[i][np.where(x[i]==max(x[i]))]=1
    print(lop)
    return lop



In [853]:
trues=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]=trues[math.floor(np.where(valy[i]==1)[0][0]/13)][np.where(valy[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
zs=np.zeros([13,13])
for i in range(len(thresx)):
    if len(np.where(thresx[i]==1)[0])==1:
        zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]=zs[math.floor(np.where(thresx[i]==1)[0][0]/13)][np.where(thresx[i]==1)[0][0]%13]+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1


In [838]:
#Our final accuracy statement
correct/len(valy)


0.2979233226837061

In [854]:
print(trues)
print(zs)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   1.  13.  18.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.  21.  44.  58.  31.   1.   0.   0.   0.   0.]
 [  0.   0.   0.  11.  34.  89.  96.  55.  27.   3.   0.   0.   0.]
 [  0.   0.   0.   3.  43. 103. 101. 105.  45.   8.   0.   0.   0.]
 [  0.   0.   0.   0.   6.  55.  88.  64.  24.   2.   0.   0.   0.]
 [  0.   0.   0.   0.   0.  16.  27.  37.   8.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   2.   4.   7.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]]
[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0. 

In [767]:
cor=0
for i in range(len(thresx)):
    if(np.where(valy[i]==1)==np.where(thresx[i]==1)):
        print(i)
        cor=cor+1
    #if i<len(thresx)-1:
        #if(np.where(trainy[i]==1)==np.where(thresx[i+1]==1)):
           # print(i)
            #cor=cor+1
    #if i>0:
        #if(np.where(trainy[i]==1)==np.where(thresx[i-1]==1)):
            #print(i)
            #cor=cor+1
cor

1
6
12
15
16
22
32
39
41
43
46
50
51
62
67
68
77
80
81
82
84
86
88
91
94
95
96
97
100
106
108
110
113
122
123
130
137
141
145
155
156
158
159
161
166
169
175
179
184
186
187
189
198
202
209
218
224
229
230
233
235
239
242
256
260
261
263
266
268
269
272
276
278
279
285
286
287
289
290
292
294
295
296
301
302
304
305
309
312
313
315
321
322
324
325
326
327
328
329
331
332
334
339
341
345
347
349
351
358
359
361
363
366
374
376
378
381
383
384
385
386
387
391
393
394
399
409
410
412
417
423
424
425
430
434
435
437
439
441
448
451
453
454
456
460
462
463
464
465
466
467
469
475
476
480
481
482
485
487
488
490
496
499
500
502
503
505
508
509
516
521
523
526
528
530
531
534
536
537
540
550
552
555
556
557
558
559
561
562
564
572
575
582
587
588
590
604
615
632
641
645
648
657
670
673
677
678
685
689
700
704
706
707
708
711
721
722
729
731
742
744
745
747
755
756
760
775
783
784
788
790
793
797
798
804
809
826
830
838
851
853
859
865
866
873
883
890
893
897
910
921
927
934
935
939
947
956
96

307

In [827]:
for i in range(len(thresx)):
    
    print(np.where(thresx[i]==1))

(array([58]),)
(array([57]),)
(array([57]),)
(array([57]),)
(array([57]),)
(array([58]),)
(array([57]),)
(array([57]),)
(array([85]),)
(array([85]),)
(array([58]),)
(array([58]),)
(array([57]),)
(array([84]),)
(array([57]),)
(array([57]),)
(array([84]),)
(array([57]),)
(array([57]),)
(array([57]),)
(array([58]),)
(array([83]),)
(array([83]),)
(array([83]),)
(array([83]),)
(array([84]),)
(array([83]),)
(array([82]),)
(array([72]),)
(array([84]),)
(array([72]),)
(array([97]),)
(array([71]),)
(array([98]),)
(array([84]),)
(array([98]),)
(array([57]),)
(array([72]),)
(array([82]),)
(array([84]),)
(array([98]),)
(array([84]),)
(array([86]),)
(array([84]),)
(array([71]),)
(array([82]),)
(array([82]),)
(array([97]),)
(array([82]),)
(array([85]),)
(array([111]),)
(array([84]),)
(array([84]),)
(array([82]),)
(array([82]),)
(array([70]),)
(array([73]),)
(array([68]),)
(array([73]),)
(array([68]),)
(array([85]),)
(array([45]),)
(array([85]),)
(array([58]),)
(array([84]),)
(array([56]),)
(array([4

In [830]:
np.where(thresx[5]==1)[0]

58